# <font color="red">**3. Correlation-Based FS - Fine-tuning Hyperparameters**</font>

**Author:** Osmar Bolivar

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go

## **1. Monthly data**

In [18]:
raw = pd.read_csv("./Data/CORR_DATASET.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Data columns (total 42 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ipc_all        178 non-null    float64
 1   lag_1          5901 non-null   float64
 2   lag_2          5901 non-null   float64
 3   lag_3          5901 non-null   float64
 4   sugar_tj       5901 non-null   float64
 5   sugar_bol      5901 non-null   float64
 6   chicken_tj     5901 non-null   float64
 7   sugar_po       5901 non-null   float64
 8   sugar_cb       5901 non-null   float64
 9   sugar_lp       5901 non-null   float64
 10  sugar_or       5901 non-null   float64
 11  oil2_bol       5901 non-null   float64
 12  sugar_sc       5901 non-null   float64
 13  lag_6          5901 non-null   float64
 14  lard_co        5901 non-null   float64
 15  oil2_tj        5901 non-null   float64
 16  rice4_su       5901 non-null   float64
 17  squash_co      5901 non-null   float64
 18

In [19]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)
df

,ipc_all,lag_1,lag_2,lag_3,sugar_tj,sugar_bol,chicken_tj,sugar_po,sugar_cb,sugar_lp,...,oil2_sc,redpepper_co,silver,rice4_lp,rice4_tj,platano_co,oil_sc,chicken_co,pineapple_co,ycorn_tj
2010-01-31,0.074472,0.263790,0.457498,0.784931,165.000000,173.290323,12.822581,174.032258,169.322581,172.580645,...,7.500000,402.50,18.217434,245.270323,250.350323,75.000000,8.329677,12.000000,600.0,67.032258
2010-02-28,0.312381,0.074472,0.263790,0.457498,157.500000,183.071429,12.714286,184.107143,181.035714,180.821429,...,7.500000,402.50,18.412194,245.577857,246.990714,75.000000,8.120000,12.000000,600.0,70.535714
2010-03-31,0.685510,0.312381,0.074472,0.263790,159.677419,180.000000,13.112903,183.322581,171.548387,170.096774,...,7.500000,402.50,18.437266,249.124516,251.576452,75.000000,8.800000,12.000000,600.0,85.032258
2010-04-30,1.215574,0.685510,0.312381,0.074472,165.000000,184.833333,13.500000,188.300000,180.600000,182.433333,...,7.500000,402.50,17.997280,247.321667,250.000000,75.666667,8.800000,12.016667,600.0,80.000000
2010-05-31,1.384706,1.215574,0.685510,0.312381,165.000000,196.451613,13.032258,196.290323,186.451613,185.806452,...,7.500000,402.50,18.499560,247.864516,248.773871,71.612903,8.800000,12.193548,600.0,87.612903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,3.844356,3.516994,3.460018,3.063085,245.000000,245.000000,13.500000,240.000000,255.000000,260.000000,...,9.560000,373.75,30.017823,257.500000,366.666667,66.670000,11.040000,17.250000,500.0,86.066667
2024-07-31,3.953707,3.844356,3.516994,3.460018,245.000000,242.161290,13.935484,240.000000,255.645161,255.951613,...,9.132258,373.75,30.018693,257.500000,375.000000,66.670000,10.730645,17.250000,500.0,85.000000
2024-08-31,5.192869,3.953707,3.844356,3.516994,245.000000,241.161290,14.000000,240.000000,245.838710,261.370968,...,9.571935,373.75,28.624296,257.500000,375.000000,66.670000,11.309677,17.250000,500.0,85.000000
2024-09-30,6.182067,5.192869,3.953707,3.844356,245.000000,242.833333,14.000000,240.000000,251.666667,270.916667,...,9.921333,373.75,30.054826,257.500000,375.000000,66.670000,11.433333,17.250000,500.0,85.000000


In [20]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [21]:
df.shape

(178, 41)

## **2. Train and Val sets**

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [23]:
#train_size = int(len(df) * 0.8)
#train, validation = df[:train_size], df[train_size:]
train, validation = train_test_split(df, test_size=0.2, random_state=42)

print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 41
Obs in validation set: 36; variables in validation set: 41


In [24]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 2011-08-31 to 2013-07-31
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ipc_all        36 non-null     float64
 1   lag_1          36 non-null     float64
 2   lag_2          36 non-null     float64
 3   lag_3          36 non-null     float64
 4   sugar_tj       36 non-null     float64
 5   sugar_bol      36 non-null     float64
 6   chicken_tj     36 non-null     float64
 7   sugar_po       36 non-null     float64
 8   sugar_cb       36 non-null     float64
 9   sugar_lp       36 non-null     float64
 10  sugar_or       36 non-null     float64
 11  oil2_bol       36 non-null     float64
 12  sugar_sc       36 non-null     float64
 13  lag_6          36 non-null     float64
 14  lard_co        36 non-null     float64
 15  oil2_tj        36 non-null     float64
 16  rice4_su       36 non-null     float64
 17  squash_co      36 non-null     float64
 18  

In [25]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
#test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

X_train = train_scaled.drop('ipc_all', axis=1)
#y_train = train_scaled['ipc_all']
y_train = train['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
#y_validation = validation_scaled['ipc_all']
y_validation = validation['ipc_all']

#X_test = test_scaled.drop('ipc_all', axis=1)

## **3. Algorithms**

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  1.022; R2:  0.538; MAE:  0.809

In [53]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=1.1, fit_intercept=True, positive=True, random_state=0) 
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Train MSE:  0.2810682313192875
Train R2:  0.9473968168460392
Train MAE:  0.39886351017462107
Validation MSE:  0.29966300527136813
Validation R2:  0.9673649066281003
Validation MAE:  0.436433716653969


In [51]:
# Define parameter grid for GridSearchCV
#alphas_ridge = np.logspace(0.1, 5, num=500)
alphas_ridge = np.arange(1.0, 7, 0.1)
param_grid = {
    'alpha': alphas_ridge,  # [1.0] 
    'fit_intercept': [True],  # [True, False] 
    'copy_X': [True], 
    'max_iter': [10000], 
    'tol': [0.0001], 
    'solver': ['auto'], 
    'positive': [True],  # [True, False]
    'random_state': [0]    
}

# Instantiate Ridge model
ridge = Ridge()
# Define GridSearchCV object
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_ridge.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_ridge.best_params_)
print("Best score: ", -grid_search_ridge.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_ridge = grid_search_ridge.best_estimator_
ridge_val_pred = best_ridge.predict(X_validation)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Best parameter:  {'alpha': 1.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 10000, 'positive': True, 'random_state': 0, 'solver': 'auto', 'tol': 0.0001}
Best score:  0.3941952014394233
Validation MSE:  0.29966300527136813
Validation R2:  0.9673649066281003
Validation MAE:  0.436433716653969


In [ ]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,1.829831
1,ycorn_tj,0.142839
2,squash_co,0.098061
3,chicken_sc,0.094585
4,oil_su,0.083888
5,rice4_tj,0.079426
6,sugar_sc,0.053478
7,sugar_cb,0.048834
8,oil_bol,0.048089
9,sugar_lp,0.047975


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.796
Validation R2:  0.640
Validation MAE:  0.689

In [62]:
#lasso = Lasso(random_state=0)
lasso = Lasso(alpha=0.1, fit_intercept=True, positive=True, max_iter=10000, random_state=0)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

Train MSE:  0.30539097049709224
Train R2:  0.9428447068556267
Train MAE:  0.4078020623645348
Validation MSE:  0.32038185574186717
Validation R2:  0.965108499905319
Validation MAE:  0.4349704518467289


In [56]:
# Define parameter grid for GridSearchCV
alphas_lasso = np.logspace(-1, 5, num=500)
#alphas_lasso = np.arange(0.1, 100, 0.05)
param_grid = {
    'alpha': alphas_lasso,  #  [1.0]
    'fit_intercept': [True], 
    'precompute': [False], 
    'copy_X': [True], 
    'max_iter': [10000],  # [1000]
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [True], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate lasso model
lasso = Lasso()
# Define GridSearchCV object
grid_search_lasso = GridSearchCV(lasso, param_grid, cv=tscv5, scoring='neg_mean_squared_error')  
# Fit GridSearchCV on training set
grid_search_lasso.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_lasso.best_params_)
print("Best score: ", -grid_search_lasso.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_lasso = grid_search_lasso.best_estimator_
y_val_pred = best_lasso.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_lasso = best_lasso.predict(X_test)

Best parameter:  {'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'max_iter': 10000, 'positive': True, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
Best score:  0.45444511236168833
Validation MSE:  0.32038185574186717
Validation R2:  0.965108499905319
Validation MAE:  0.4349704518467289


In [63]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,1.933709
1,sugar_lp,0.116314
2,rice4_su,0.092692
3,ycorn_tj,0.092405
4,rice4_tj,0.034357
5,chicken_sc,0.023554
6,oil_su,0.019175
7,oil2_sc,0.000000
8,apple_tr,0.000000
9,lard_or,0.000000


### **3.3. ElasticNet**
Without tuning:
Validation MSE:  0.2237
Validation R2:  0.250
Validation MAE:  0.401

In [64]:
enet = ElasticNet(random_state=0)
#enet = ElasticNet(alpha=0.4, l1_ratio=0.97, fit_intercept=True, max_iter=10000, random_state=0) 
# Fit on training set
enet.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
enet_train_pred = enet.predict(X_train)
enet_val_pred = enet.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_enet = mean_squared_error(y_train, enet_train_pred)
r2_train_enet = r2_score(y_train, enet_train_pred)
mae_train_enet = mean_absolute_error(y_train, enet_train_pred)
print("Train MSE: ", mse_train_enet)
print("Train R2: ", r2_train_enet)
print("Train MAE: ", mae_train_enet)
# Calculate Forecast metrics on validation set
mse_val_enet = mean_squared_error(y_validation, enet_val_pred)
r2_val_enet = r2_score(y_validation, enet_val_pred)
mae_val_enet = mean_absolute_error(y_validation, enet_val_pred)
print("Validation MSE: ", mse_val_enet)
print("Validation R2: ", r2_val_enet)
print("Validation MAE: ", mae_val_enet)

Train MSE:  1.1181273987926925
Train R2:  0.790737430295568
Train MAE:  0.7957355659973951
Validation MSE:  1.6037902243795656
Validation R2:  0.8253376532943418
Validation MAE:  0.8868445665207338


In [ ]:
# Define parameter grid for GridSearchCV
#alphas_enet = np.logspace(-1, 2, num=500)
alphas_enet = np.arange(0.1, 10, 0.05)
param_grid = {
    'alpha': [1.0], # alphas_enet
    'l1_ratio': [0.5], # np.arange(0.1, 0.95, 0.05)
    'fit_intercept': [True], 
    'precompute': [False], 
    'max_iter': [10000], 
    'copy_X': [True], 
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [False], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate enet model
enet = ElasticNet()
# Define GridSearchCV object
grid_search_enet = GridSearchCV(enet, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_enet.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_enet.best_params_)
print("Best score: ", -grid_search_enet.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_enet = grid_search_enet.best_estimator_
y_val_pred = best_enet.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_enet = best_enet.predict(X_test)

In [65]:
# Get the coefficients from the enet model
coef = enet.coef_
# Create a dataframe of feature importances
feature_importance_enet = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_enet = feature_importance_enet.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_enet.columns = ['feat_enet', 'imp_enet']
# Print the feature importances
feature_importance_enet.head(15)

,feat_enet,imp_enet
0,lag_1,0.646511
1,lag_2,0.354198
2,lag_3,0.155714
3,oil2_bol,0.091620
4,sugar_lp,0.054239
5,chicken_tj,0.051754
6,sugar_sc,0.047706
7,sugar_cb,0.044248
8,sugar_bol,0.040181
9,squash_co,0.036666


### **3.4. ADA**
Without tuning:
Validation MSE:  0.410
Validation R2:  0.814
Validation MAE:  0.502

In [94]:
#ada = AdaBoostRegressor(random_state=0)
ada = AdaBoostRegressor(learning_rate=1.1, loss='square', n_estimators=55, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

Train MSE:  0.14226046790467434
Train R2:  0.9733753138388066
Train MAE:  0.3044844431072564
Validation MSE:  0.4751256162089367
Validation R2:  0.9482559789643757
Validation MAE:  0.5462773406895488


In [ ]:
# Define the AdaBoost Regressor
ada = AdaBoostRegressor()
# Define the range of hyperparameters to search over
param_grid_ada = {
    'estimator': [DecisionTreeRegressor(max_depth=3),  # [None]
                  DecisionTreeRegressor(max_depth=4),
                  DecisionTreeRegressor(max_depth=5),
                  DecisionTreeRegressor(max_depth=6),
                  DecisionTreeRegressor(max_depth=7),
                  DecisionTreeRegressor(max_depth=8) ],   
    'n_estimators': [50],  # range(50, 200, 5)
    'learning_rate': [0.99], # np.logspace(-5,1, 100)
    'loss': ['square'], # ['linear']
    'random_state': [0]
}

# Tune hyperparameters using GridSearchCV with TimeSeriesSplit
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=tscv5, scoring='neg_mean_squared_error')
grid_search_ada.fit(X_train, y_train)
# Evaluate the model using the best hyperparameters on the test set
ada_best = AdaBoostRegressor(**grid_search_ada.best_params_)
ada_best.fit(X_train, y_train)
# Make predictions on the validation set
y_val_pred = ada_best.predict(X_validation)
# Evaluate the model on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Print the best hyperparameters and the best score
print("Best parameters found: ", grid_search_ada.best_params_)
print("Lowest MSE found: ", -grid_search_ada.best_score_)
# Evaluate the model on the test set
#y_test_pred_ada = ada_best.predict(X_test)

In [95]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
# Print the feature importance DataFrame
feature_importance_ada.head(15)

,feat,imp_ada
0,lag_1,0.730560
1,chicken_tj,0.050027
2,silver,0.041230
3,sugar_tj,0.039859
4,sugar_or,0.026699
5,chicken_sc,0.022907
6,sugar_cb,0.018183
7,ycorn_tj,0.009473
8,sugar_lp,0.008047
9,platano_co,0.006613


### **3.5. GBR**
Without tuning:
Validation MSE:  0.423
Validation R2:  0.808
Validation MAE:  0.522

In [117]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(learning_rate=0.5, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

Train MSE:  1.6884566881523766e-06
Train R2:  0.9999996839977396
Train MAE:  0.0010646627240440813
Validation MSE:  0.3583244258605745
Validation R2:  0.960976327108505
Validation MAE:  0.4665268893314685


In [ ]:
# Define the model
gbr = GradientBoostingRegressor()
# Define the hyperparameters to be tuned
params = {
    'loss': ['squared_error'], # ['squared_error', 'absolute_error', 'huber', 'quantile']
    'learning_rate': [0.2], # [0.1] 
    'n_estimators': range(100, 200, 1),  # [110]
    'subsample': [1.0], 
    'criterion': ['friedman_mse'], # ['friedman_mse', 'squared_error']
    'min_samples_split': [2], 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_depth': [4], # range(3,6,1)
    'min_impurity_decrease': [0.0], 
    'init': [None], 
    'random_state': [0], 
    'max_features': [None], 
    'alpha': [0.9], 
    'verbose': [0], 
    'max_leaf_nodes': [None], 
    'warm_start': [False], 
    'validation_fraction': [0.1], 
    'n_iter_no_change': [None], 
    'tol': [0.0001], 
    'ccp_alpha': [0.0]
}

# Create the GridSearchCV object
grid_gbr = GridSearchCV(gbr, params, cv=tscv5, scoring='neg_mean_squared_error', n_jobs=-1)
# Fit the model on the training set with GridSearchCV
grid_gbr.fit(X_train, y_train)
# Print the best hyperparameters
print('Best hyperparameters:', grid_gbr.best_params_)
# Use the best model to make predictions on the validation set
y_val_pred = grid_gbr.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Use the best model to make predictions on the test set
#y_test_pred_gbr = grid_gbr.predict(X_test)

In [118]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
# Print the feature importance DataFrame
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,lag_1,0.806133
1,chicken_tj,0.116398
2,sugar_lp,0.010525
3,sugar_cb,0.009272
4,ycorn_tj,0.008552
5,watermelon_po,0.007798
6,chicken_sc,0.006211
7,lag_6,0.005301
8,apple_tr,0.005067
9,sugar_or,0.003286


### **3.4. RF**
Without tuning:
Validation MSE:  0.597
Validation R2:  0.730
Validation MAE:  0.657

In [126]:
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(criterion="absolute_error", max_depth=5, random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

Train MSE:  0.08471345642906626
Train R2:  0.9841454957636919
Train MAE:  0.21579590278937213
Validation MSE:  0.40358078408956805
Validation R2:  0.9560476390472681
Validation MAE:  0.5021545319026356


In [ ]:
# Define the Random Forest Regression model
rf_reg = RandomForestRegressor()

# Define the hyperparameters to tune
param_grid_rf = {
    'n_estimators': range(100, 300, 10),  # [100]
    'criterion': ['absolute_error'], 
    'max_depth': [7], # [None] 
    'min_samples_split': [2],  # range(2,20,1) 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_features': [1.0], 
    'max_leaf_nodes': [None], 
    'min_impurity_decrease': [0.0], 
    'bootstrap': [True], 
    'oob_score': [False], 
    'n_jobs':[None], 
    'random_state': [0], 
    'verbose': [0], 
    'warm_start': [False], 
    'ccp_alpha': [0.0], 
    'max_samples': [None], 
    'monotonic_cst': [None]
}

# Define the GridSearchCV object
grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=tscv5, scoring='neg_mean_squared_error')
# Fit the GridSearchCV object to the training data
grid_rf_reg.fit(X_train, y_train)
# Extract the best hyperparameters and score
best_params = grid_rf_reg.best_params_
best_score = grid_rf_reg.best_score_
# Print the best hyperparameters found by GridSearchCV
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")
# Instantiate a new Random Forest Regression model using the best hyperparameters
rf_reg_best = RandomForestRegressor(**best_params)
# Fit the model to the training data and make predictions on the validation set
rf_reg_best.fit(X_train, y_train)
# Use the best model to make predictions on the validation set
y_val_pred = rf_reg_best.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_rf = rf_reg_best.predict(X_test)

In [127]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
# Print the feature importance DataFrame
feature_importance_rf.head(30)

,feat,imp_rf
0,lag_1,0.754556
1,chicken_tj,0.018980
2,ycorn_tj,0.016413
3,sugar_tj,0.013353
4,sugar_cb,0.012017
5,lag_6,0.010706
6,lag_2,0.010464
7,lag_3,0.010137
8,lard_co,0.009848
9,chicken_sc,0.009331


### **3.5. ET**   
Without tuning:
Validation MSE:  1.453
Validation R2:  0.342
Validation MAE:  1.032

In [141]:
#et = ExtraTreesRegressor(random_state=0)
et = ExtraTreesRegressor(max_depth=8, bootstrap=True, oob_score=True, random_state=0)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

Train MSE:  0.050856961415745826
Train R2:  0.9904818910218018
Train MAE:  0.16582903461996445
Validation MSE:  0.3342800793132508
Validation R2:  0.9635949002417739
Validation MAE:  0.4747834737941856


In [ ]:
# Define the Extra Trees Regression model
et_reg = ExtraTreesRegressor()
# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [100], # range(100, 300, 15)
     'criterion': ['squared_error'], 
     'max_depth':[None], 
     'min_samples_split':[2], 
     'min_samples_leaf':[1], 
     'min_weight_fraction_leaf':[0.0], 
     'max_features':[1.0], 
     'max_leaf_nodes':[None], 
     'min_impurity_decrease':[0.0], 
     'bootstrap':[True], # [False] 
     'oob_score':[True], # [False]
     'n_jobs':[None], 
     'random_state':[0], 
     'verbose':[0], 
     'warm_start':[False], 
     'ccp_alpha':[0.0], 
     'max_samples':[None],   
     'monotonic_cst':[None]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=tscv5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
# Print the best hyperparameters and the corresponding validation score
print("Best hyperparameters: ", grid_search.best_params_)
#print("Validation score: ", grid_search.best_score_)
# Use the best model to make predictions on the validation set
best_et_reg = grid_search.best_estimator_
y_val_pred = best_et_reg.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_et = best_et_reg.predict(X_test)

In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
feature_importance_et.head(30)

,feat,imp_et
0,lag_1,0.328512
1,lag_2,0.175129
2,lag_3,0.051503
3,rice4_su,0.025076
4,lard_co,0.023527
5,sugar_tj,0.020728
6,lag_6,0.020710
7,chicken_tj,0.019826
8,sugar_po,0.019540
9,onion2_tj,0.019471


#### End